# Approximating square root iteratively

There are a lot of useful functions that have no closed form solution, meaning we can't just do a computation and return the value. Instead, we need to use an iterative method to approximate the function value. It's an extremely useful pattern that will come in handy later in this program and in your professional life. We can use iterative computing to approximate sine (with Taylor series expansion), approximate square root (as we'll do in this lecture), or optimize a cost or error function (gradient descent in the introduction to machine learning course).

The goal of this lab is to translate a recurrence relation, that yields a sequence of values, to Python.  Instead of computing and returning a single value, we will look for **convergence of the series**. In other words, if we run the series out far enough, $x_{i+1}$ will be close to $x_i$ leaving $x_i$ as  a very accurate approximation of square root. This will teach us the basics of iterative computing and prepare us for  more complicated function optimization material needed later.

## Babylonian method

To approximate square root, the idea is to pick an initial estimate, $x_0$, and then iterate with better and better estimates, $x_i$, using the ([Babylonian method](https://en.wikipedia.org/wiki/Methods_of_computing_square_roots#Babylonian_method)) recurrence relation:

$x_{i+1} = \frac{1}{2}(x_i + \frac{n}{x_i})$

There’s a great deal on the web you can read to learn more about why this process works but it relies on the average (midpoint) of $x_i$ and $n/x_i$ getting us closer to the square root of n. The cool thing is that the iteration converges quickly.

Our goal is to write a function that takes a single number and returns its square root. What do we know about this function before even beginning to code? Well, we have a clear description of the problem per the recurrence relation, and we also have the function signature we want:

```python
def sqrt(n):
```

Because we are implementing a recurrence relation, we know that we will have a loop that computes $x_{i+1}$ from $x_{i}$. 

### Convergence

The terminating condition of the loop is when we have reached convergence or close to it. Convergence just means that $x_{i+1}$ is pretty close to $x_i$. Because we can never compare to real numbers for equality, we have to check for the difference being smaller than some precision like 0.00000001.

### Iterative method outline

Just as we have an outline for a data science program, iterative methods all share the same basic outline. (I'm assuming here that $x_{i+1}$ depends only on a single previous value and that $i$ implicitly increments as the loop goes around.)

*set $x_0$ to initial value*<br>
*repeat:*<br>
&nbsp;&nbsp;&nbsp;&nbsp;$x_{i+1} =$ function-giving-next-value$(x_i)$<br>
*until $abs(x_{i+1} - x_i) \lt precision$<br>
return $x_{i+1}$*<br>

Because Python does not have a repeat-until loop, we fake it with an infinite loop containing a conditional that breaks us out upon convergence:

*set $x_0$ to initial value*<br>
*while True:*<br>
&nbsp;&nbsp;&nbsp;&nbsp;$x_{i+1} =$ function-giving-next-value$(x_i)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;*if $abs(x_{i+1} - x_i) \lt precision$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return $x_{i+1}$*<br>

That is a fairly straightforward implementation of the recurrence relation, but you will notice that we don't actually need to keep all previous $x_i$ around except for the new value and the previous value. Here is a Python implementation that tracks only two values and follows the infinite loop pattern:

In [1]:
def sqrt(n):
    "compute square root of n"
    PRECISION = 0.00000001 # stop iterating when we converge with this delta
    x_0 = 1.0 # pick any old initial value
    x_prev = x_0
    while True: # Python doesn't have repeat-until loop so fake it
        #print(x_prev)
        x_new = 0.5 * (x_prev + n/x_prev)
        if abs(x_new - x_prev) < PRECISION:
            return x_new
        x_prev = x_new # x_i+1 becomes x_i (previous value)

In [2]:
sqrt(100)

10.0

To test our square root approximation, we can compare it to `math.sqrt()` and use numpy's `isclose` to do the comparison.

In [3]:
import numpy as np

def check(n):
    assert np.isclose(sqrt(n), np.sqrt(n))

def test_big():
    check(125348)

def test_medium():
    check(89.2342)

def test_100():
    check(100)

def test_1():
    check(1)

def test_0():
    check(0)

test_big()
test_medium()
test_100()
test_1()
test_0()

### Exercise

Type in (don't cut/paste) the `sqrt(n)` function and test with, for example, `sqrt(125348.0)`. Make sure you get the right answer (354.045195) and then add print statements so that you can see the sequence of $x_{i}$ values. I get:

```
1.0
62674.5
31338.249992
15671.1249162
7839.56178812
3927.77547356
1979.84435152
1021.5781996
572.139273508
395.612894667
356.228988269
354.051888518
354.045194918
354.045194855
```

Notice how quickly it converges!

In [4]:
def sqrt_with_trace(n):
    "compute square root of n"
    PRECISION = 0.00000001 # stop iterating when we converge with this delta
    x_0 = 1.0 # pick any old initial value
    x_prev = x_0
    while True: # Python doesn't have repeat-until loop so fake it
        print(x_prev)
        x_new = 0.5 * (x_prev + n/x_prev)
        if abs(x_new - x_prev) < PRECISION:
            return x_new
        x_prev = x_new

sqrt_with_trace(125348.000000)

1.0
62674.5
31338.249992022273
15671.12491623693
7839.561788117747
3927.7754735639414
1979.844351517673
1021.5781996033152
572.1392735077299
395.612894666841
356.2289882689982
354.0518885182295
354.04519491839494
354.04519485512014


354.04519485512014

### Exercise: Testing with pytest

 To run with the unit tester pytest, save the `sqrt()` function in a file called `sqrt.py` and then put the test code into a file called `test_sqrt.py`.  That test file will also need to import the square root code so add
 
```
from sqrt import *
```

to the top of the test file. To run the tests from the command line, you can do the following:

```bash
$ python test_sqrt.py 
$
```

which prints nothing because there are no errors (hopefully).

A better way is to use `pytest` which executes all functions that start with prefix `test_`:

```bash
$ pip install pytest
$ pytest -v test_sqrt.py 
============================= test session starts ==============================
platform darwin -- Python 3.8.8, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /Users/parrt/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /private/tmp
plugins: anyio-2.2.0
collected 5 items                                                              

test_sqrt.py::test_big PASSED                                            [ 20%]
test_sqrt.py::test_medium PASSED                                         [ 40%]
test_sqrt.py::test_100 PASSED                                            [ 60%]
test_sqrt.py::test_1 PASSED                                              [ 80%]
test_sqrt.py::test_0 PASSED                                              [100%]

============================== 5 passed in 0.07s ===============================
```

The other benefit is that we don't need the explicit calls so you can delete this part and it still works with pytest:
 
```python
test_big()
test_medium()
test_100()
test_1()
test_0()
```